# Heuristicas

In [3]:
import utils
import pandas as pd
import seaborn as sns

DATA = "../data/sim.dat"
POWERS = "../data/sim_powers.tsv"

In [4]:
def read_powers() -> pd.DataFrame:
    return pd.read_csv(
        POWERS,
        sep=' ',
        header=None,
        names=["team", "power"],
    )

def read_sim_ranking(method: str) -> pd.DataFrame:
    powers = read_powers()
    ranking = utils.read_ranking(DATA+method+utils.OUT)
    ranking["power"] = (ranking.index + 1).map(powers.set_index("team")["power"])
    ranking["team"] = (ranking.index+1)
    return ranking

In [ ]:
utils.run(DATA, utils.METHOD_CMM)
utils.run(DATA, utils.METHOD_WP)

In [ ]:
read_sim_ranking(utils.METHOD_WP).sort_values("rating", ascending=False)

In [ ]:
utils.analyze_data(DATA)

In [5]:
import simulator

TEAM = 20
ITER = 500
ratings = []

#for i in range(ITER):
#if i % 100 == 0: print(i)

simulator.simulate(team_count=50)
utils.run(DATA, utils.METHOD_CMM)
ranking = read_sim_ranking(utils.METHOD_WP).sort_values("rating", ascending=False)
#ratings.append(ranking.set_index("team").at[TEAM, "rating"])
ranking.set_index("team")
#sns.boxplot(ratings)

team_count: 50


,rating,power
team,,
10,0.888889,20.0
9,0.870370,18.0
7,0.740741,14.0
8,0.703704,16.0
6,0.537037,12.0
5,0.481481,10.0
4,0.314815,8.0
3,0.259259,6.0
2,0.185185,4.0


In [ ]:
def heuristic(games: List[simulator.Game]) -> List[simulator.Game]:
    # calculas cmm
    # run()
    # modificas los partididos segun tu heuristica
    # run()
    # ves el ranking
    